# San Francisco Crime

## Leo CSV como RDD

In [1]:
dataframe = sqlContext.read.csv('../../data/sf.csv', header=True)

In [2]:
data = dataframe.rdd
data

MapPartitionsRDD[12] at javaToPython at NativeMethodAccessorImpl.java:0

## Primer registro de los datos

In [3]:
data.first()

Row(Dates=u'2015-05-13 23:53:00', Category=u'WARRANTS', Descript=u'WARRANT ARREST', DayOfWeek=u'Wednesday', PdDistrict=u'NORTHERN', Resolution=u'ARREST, BOOKED', Address=u'OAK ST / LAGUNA ST', X=u'-122.425891675136', Y=u'37.7745985956747')

## Cantidad de registros

In [4]:
data.count()

878049

## Cantidad de categorías de crímenes

In [12]:
%%time
categories = data.map(lambda x: (x.Category)).distinct()

CPU times: user 6.5 ms, sys: 13 ms, total: 19.5 ms
Wall time: 67.4 ms


In [10]:
%%time
data.map(lambda x: (x.Category, 1)).reduceByKey(lambda a,b: a).count()

CPU times: user 23.1 ms, sys: 2.51 ms, total: 25.6 ms
Wall time: 23.3 s


39

## Categorías de crímenes

In [13]:
categories.collect()

[u'WEAPON LAWS',
 u'WARRANTS',
 u'LOITERING',
 u'EMBEZZLEMENT',
 u'DRIVING UNDER THE INFLUENCE',
 u'SEX OFFENSES FORCIBLE',
 u'ROBBERY',
 u'BURGLARY',
 u'PORNOGRAPHY/OBSCENE MAT',
 u'RECOVERED VEHICLE',
 u'BRIBERY',
 u'BAD CHECKS',
 u'SUSPICIOUS OCC',
 u'ARSON',
 u'PROSTITUTION',
 u'VANDALISM',
 u'LIQUOR LAWS',
 u'TREA',
 u'VEHICLE THEFT',
 u'ASSAULT',
 u'FAMILY OFFENSES',
 u'RUNAWAY',
 u'EXTORTION',
 u'KIDNAPPING',
 u'MISSING PERSON',
 u'DISORDERLY CONDUCT',
 u'SECONDARY CODES',
 u'STOLEN PROPERTY',
 u'FORGERY/COUNTERFEITING',
 u'TRESPASS',
 u'SUICIDE',
 u'DRUNKENNESS',
 u'FRAUD',
 u'NON-CRIMINAL',
 u'GAMBLING',
 u'DRUG/NARCOTIC',
 u'OTHER OFFENSES',
 u'SEX OFFENSES NON FORCIBLE',
 u'LARCENY/THEFT']

## Categorías de crímenes ordenadas alfabeticamente

In [14]:
categories.takeOrdered(40)

[u'ARSON',
 u'ASSAULT',
 u'BAD CHECKS',
 u'BRIBERY',
 u'BURGLARY',
 u'DISORDERLY CONDUCT',
 u'DRIVING UNDER THE INFLUENCE',
 u'DRUG/NARCOTIC',
 u'DRUNKENNESS',
 u'EMBEZZLEMENT',
 u'EXTORTION',
 u'FAMILY OFFENSES',
 u'FORGERY/COUNTERFEITING',
 u'FRAUD',
 u'GAMBLING',
 u'KIDNAPPING',
 u'LARCENY/THEFT',
 u'LIQUOR LAWS',
 u'LOITERING',
 u'MISSING PERSON',
 u'NON-CRIMINAL',
 u'OTHER OFFENSES',
 u'PORNOGRAPHY/OBSCENE MAT',
 u'PROSTITUTION',
 u'RECOVERED VEHICLE',
 u'ROBBERY',
 u'RUNAWAY',
 u'SECONDARY CODES',
 u'SEX OFFENSES FORCIBLE',
 u'SEX OFFENSES NON FORCIBLE',
 u'STOLEN PROPERTY',
 u'SUICIDE',
 u'SUSPICIOUS OCC',
 u'TREA',
 u'TRESPASS',
 u'VANDALISM',
 u'VEHICLE THEFT',
 u'WARRANTS',
 u'WEAPON LAWS']

## Las 10 categorías de crímenes con más casos

In [16]:
%%time
data.map(lambda x: (x.Category, 1))\
    .reduceByKey(lambda x,y: x+y)\
    .takeOrdered(1, key = lambda x: -x[1])

CPU times: user 19.8 ms, sys: 7.95 ms, total: 27.8 ms
Wall time: 23.3 s


[(u'LARCENY/THEFT', 174900)]

## La categoría con más casos

In [17]:
%%time
data.map(lambda x: (x.Category, 1))\
    .reduceByKey(lambda x,y: x+y)\
    .reduce(lambda x,y: x if x[1]>y[1] else y)

CPU times: user 29.8 ms, sys: 3.24 ms, total: 33 ms
Wall time: 22.2 s


(u'LARCENY/THEFT', 174900)

## Día de la semana con más casos de 'Driving under the influence'

In [19]:
%%time
data.filter(lambda x: x.Category == 'DRIVING UNDER THE INFLUENCE')\
    .map(lambda x: (x.DayOfWeek, 1))\
    .reduceByKey(lambda x,y: x+y)\
    .takeOrdered(7, key=lambda x: -x[1])

CPU times: user 21.2 ms, sys: 5.75 ms, total: 26.9 ms
Wall time: 23.9 s


[(u'Saturday', 457),
 (u'Sunday', 442),
 (u'Friday', 352),
 (u'Thursday', 282),
 (u'Monday', 263),
 (u'Tuesday', 251),
 (u'Wednesday', 221)]

In [20]:
%%time
data.map(lambda x: ((x.DayOfWeek, x.Category), 1))\
    .reduceByKey(lambda x,y: x+y)\
    .filter(lambda x: x[0][1] == 'DRIVING UNDER THE INFLUENCE')\
    .takeOrdered(7, key=lambda x: -x[1])

CPU times: user 30.2 ms, sys: 3.2 ms, total: 33.4 ms
Wall time: 24.4 s


[((u'Saturday', u'DRIVING UNDER THE INFLUENCE'), 457),
 ((u'Sunday', u'DRIVING UNDER THE INFLUENCE'), 442),
 ((u'Friday', u'DRIVING UNDER THE INFLUENCE'), 352),
 ((u'Thursday', u'DRIVING UNDER THE INFLUENCE'), 282),
 ((u'Monday', u'DRIVING UNDER THE INFLUENCE'), 263),
 ((u'Tuesday', u'DRIVING UNDER THE INFLUENCE'), 251),
 ((u'Wednesday', u'DRIVING UNDER THE INFLUENCE'), 221)]

## Cuales son los distritos?

In [22]:
data.map(lambda x: (x.PdDistrict)).distinct().collect()

[u'CENTRAL',
 u'NORTHERN',
 u'SOUTHERN',
 u'PARK',
 u'TENDERLOIN',
 u'RICHMOND',
 u'TARAVAL',
 u'BAYVIEW',
 u'INGLESIDE',
 u'MISSION']

## Cuales son las resolutions?

In [24]:
data.map(lambda x: (x.Resolution)).distinct().collect()

[u'NONE',
 u'JUVENILE DIVERTED',
 u'COMPLAINANT REFUSES TO PROSECUTE',
 u'DISTRICT ATTORNEY REFUSES TO PROSECUTE',
 u'PSYCHOPATHIC CASE',
 u'PROSECUTED FOR LESSER OFFENSE',
 u'JUVENILE ADMONISHED',
 u'ARREST, CITED',
 u'NOT PROSECUTED',
 u'ARREST, BOOKED',
 u'EXCEPTIONAL CLEARANCE',
 u'UNFOUNDED',
 u'LOCATED',
 u'JUVENILE CITED',
 u'JUVENILE BOOKED',
 u'PROSECUTED BY OUTSIDE AGENCY',
 u'CLEARED-CONTACT JUVENILE FOR MORE INFO']

## Categoría con mayor porcentaje de resolución “Not Prosecuted”

In [25]:
data.map(lambda x: (x.Category, (1, int(x.Resolution == 'NOT PROSECUTED'))))\
    .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
    .takeOrdered(3, key=lambda x: -(float(x[1][1])/x[1][0]))

[(u'FORGERY/COUNTERFEITING', (10609, 698)),
 (u'FRAUD', (16679, 946)),
 (u'BAD CHECKS', (406, 19))]

## Spark SQL

In [ ]:
dataframe.registerTempTable("crimes")
sqlCtx.sql("select distinct Category from crimes").collect()

In [ ]:
sqlCtx.sql("select distinct Category from crimes order by Category").collect()

## 10 delitos más comunes con SQL

In [ ]:
sql = "select Category, count(*) as crimes_count from crimes "
sql += "group by Category order by crimes_count desc limit 10"
sqlCtx.sql(sql).collect()

## Día de la semana con más casos de 'Driving under the influence'

In [ ]:
sql = "select DayOfWeek, count(*) as crimes_count from crimes "
sql += "where Category == 'DRIVING UNDER THE INFLUENCE'"
sql += " group by DayOfWeek order by crimes_count desc limit 1"
sqlCtx.sql(sql).collect()